# Import Modules and Define Helper Functions

In [18]:
from arcgis.gis import GIS
import pandas as pd

def adjust_url(service):
    # Adjust the service URL to replace "admin" with "rest" and format the URL correctly
    url = service.url.replace("admin", "rest").replace(f".{service.properties.type}", f"/{service.properties.type}")
    return url
    
def get_stopped_services(folder):
    # Initialize an empty list to store services that are stopped
    delete_list = []
    exempt_folders = ['Hosted', 'Utilities', 'System']
    
    print("Adding the following stopped services to a delete list")
    # Loop through services in the specified folder
    for service in ags.services.list(folder):
        # Check if the service state is 'STOPPED'
        state = service.status['realTimeState']
        if state == 'STOPPED':
            print(service)
            # Add the folder and service to the delete list
            delete_list.append([folder, service])
    return delete_list

def get_service_info(folder, service):
    # Extract and return detailed information about the service
    server_folder = folder
    name = service.properties.serviceName
    state = service.status['realTimeState']
    service_capabilities = service.properties['capabilities']
    manifest = service.iteminformation.manifest
    url = adjust_url(service)
    
    database = None
    server = None
    db_user = None 
    # If the service uses databases, extract connection details
    if manifest['databases']:
        conn_string = manifest['databases'][0]['onServerConnectionString']
        server = conn_string.split('INSTANCE=sde:sqlserver:')[-1].split(';')[0]
        database = conn_string.split('DATABASE=')[-1].split(';')[0]
        db_user = conn_string.split(';USER=')[-1].split(';')[0]
    
    authoring_doc = None
    author = None
    # If the service has resources, extract authoring document and author
    if manifest['resources']:
        authoring_doc = manifest['resources'][0]['onPremisePath']
        author = manifest['resources'][0]['clientName']
    
    feature_access = None
    feature_service_capabilities = None
    # If the service is a MapServer, check for FeatureServer extension and its capabilities
    if service.properties.type == 'MapServer':
        for extension in service.properties.extensions:
            if extension['typeName'] == 'FeatureServer':
                if extension['enabled'] == 'true':
                    feature_access = 'Yes'
                    feature_service_capabilities = extension['capabilities']
         
    # Return the extracted information as a list
    return [server_folder, name, state, url,service_capabilities, feature_access, feature_service_capabilities, 
            database, db_user, server, authoring_doc, author]

def delete_services(list):
    # Loop through the list of stopped services and delete each one
    for folder, service in list:
        '''MAKE SURE TO UNCOMMENT THE LINE BELOW WHEN READY TO DELETE'''
        #service.delete()
        print(f"{service.properties.serviceName} DELETED")

# Connect to GIS

In [ ]:
# Connect to your GIS - Input your portal URL, admin username, and password
gis = GIS('portal url', 'admin user', 'admin password')

# If you only have one ArcGIS Server, the index position will always be 0. 
# If you have more than one, the Host will be at index 0 followed by other servers.
ags = gis.admin.servers.list()[0]

# Grab services to delete - services must be stopped

In [12]:
# Get the list of stopped services in a folder
delete_list = get_stopped_services('Tmp')

Adding the following stopped services to a delete list
<Service at https://devGISweb.halff.com/agsdev/admin/services/Tmp/Bedford_Map_Templates_test.GPServer>
<Service at https://devGISweb.halff.com/agsdev/admin/services/Tmp/Broadband_Inspection_Viewer_FeatureClasses.MapServer>
<Service at https://devGISweb.halff.com/agsdev/admin/services/Tmp/FloodHazards_test_12_01.MapServer>
<Service at https://devGISweb.halff.com/agsdev/admin/services/Tmp/Line1.MapServer>
<Service at https://devGISweb.halff.com/agsdev/admin/services/Tmp/NativeSQLPoly.MapServer>
<Service at https://devGISweb.halff.com/agsdev/admin/services/Tmp/Poly1.MapServer>
<Service at https://devGISweb.halff.com/agsdev/admin/services/Tmp/SQLGeogpt.MapServer>
<Service at https://devGISweb.halff.com/agsdev/admin/services/Tmp/SQLGeom.MapServer>
<Service at https://devGISweb.halff.com/agsdev/admin/services/Tmp/SQLGeomTesting.MapServer>
<Service at https://devGISweb.halff.com/agsdev/admin/services/Tmp/WR_Polygon.MapServer>
<Service at 

# Create and export your archive file - Adjust folder_dir variable

In [17]:
# Initialize an empty list to store service information
archive = []
# Loop through the delete list and get detailed information for each service
for folder, service in delete_list:
    archive.append(get_service_info(folder, service))

# Create a DataFrame from the collected service information
df = pd.DataFrame(data=archive, columns=['Folder Name', 'Name', 'State', 'Map Service URL', 'Service Capabilities', 
                                         "Feature Access", "Feature Capabilities", 'Database', 
                                         'DB User', 'Server', 'Publishing Document', 'Authoring Computer Name'])

# Define the folder and name for the Excel file to store the archive
folder_dir = r"C:\jlopez\ExcelReport"


file_name = f"{folder} Services Archive"
archive_file = f"{folder_dir}\\{file_name}.xlsx"

# Save the DataFrame to an Excel file
df.to_excel(archive_file, index=False)
# Print the contents of the DataFrame
df.head(len(df))

,Folder Name,Name,State,Map Service URL,Service Capabilities,Feature Access,Feature Capabilities,Database,DB User,Server,Publishing Document,Authoring Computer Name
0,Tmp,Bedford_Map_Templates_test,STOPPED,https://devGISweb.halff.com/agsdev/rest/servic...,,None,None,None,None,None,C:\Users\ah4236\AppData\Local\Temp\ArcGISProTe...,Lopez-3291
1,Tmp,Broadband_Inspection_Viewer_FeatureClasses,STOPPED,https://devGISweb.halff.com/agsdev/rest/servic...,"Map,Query,Data",Yes,"Create,Query,Update,Delete,Uploads,Editing",Broadband,sde,dev-bld-sql19\sde,C:\Users\ah4236\AppData\Local\Temp\ArcGISProTe...,Lopez-3291
2,Tmp,FloodHazards_test_12_01,STOPPED,https://devGISweb.halff.com/agsdev/rest/servic...,"Query,Map,Data",None,None,NFHL,GISPublisher,rchdb02-ahv\sde,W:\Citrix\Misc\GIS Admin Projects\NFHL\NFHL.aprx,Lopez-3291
3,Tmp,Line1,STOPPED,https://devGISweb.halff.com/agsdev/rest/servic...,"Query,Map,Data",Yes,"Query,Create,Update,Delete,Editing,Uploads",JoelArcGISImport,GISPublisher,dev-bld-sql19\sde,C:\Users\ah4236\AppData\Local\Temp\ArcGISProTe...,Lopez-3291
4,Tmp,NativeSQLPoly,STOPPED,https://devGISweb.halff.com/agsdev/rest/servic...,"Query,Map,Data",Yes,"Query,Create,Update,Delete,Editing,Uploads",GrandParkwayTracker,GISPublisher,dev-bld-sql19\sde,W:\Citrix\Misc\GIS Admin Projects\NativeSQL\Na...,Lopez-3291
5,Tmp,Poly1,STOPPED,https://devGISweb.halff.com/agsdev/rest/servic...,"Query,Map,Data",Yes,"Query,Create,Update,Delete,Editing,Uploads",JoelArcGISImport,GISPublisher,dev-bld-sql19\sde,C:\Users\ah4236\AppData\Local\Temp\ArcGISProTe...,Lopez-3291
6,Tmp,SQLGeogpt,STOPPED,https://devGISweb.halff.com/agsdev/rest/servic...,"Query,Map,Data",Yes,"Query,Create,Update,Delete,Editing,Uploads",JoelArcGISImport,GISPublisher,dev-bld-sql19\sde,C:\Users\ah4236\AppData\Local\Temp\2\ArcGISPro...,CTX22APP02-AHV
7,Tmp,SQLGeom,STOPPED,https://devGISweb.halff.com/agsdev/rest/servic...,"Query,Map,Data",Yes,"Query,Create,Update,Delete,Editing,Uploads",JoelArcGISImport,GISPublisher,dev-bld-sql19\sde,C:\Users\ah4236\AppData\Local\Temp\2\ArcGISPro...,CTX22APP02-AHV
8,Tmp,SQLGeomTesting,STOPPED,https://devGISweb.halff.com/agsdev/rest/servic...,"Query,Map,Data",Yes,"Query,Create,Update,Delete,Editing,Uploads",JoelArcGISImport,GISPublisher,dev-bld-sql19\sde,C:\Users\ah4236\AppData\Local\Temp\2\ArcGISPro...,CTX22APP02-AHV
9,Tmp,WR_Polygon,STOPPED,https://devGISweb.halff.com/agsdev/rest/servic...,"Map,Query,Data",Yes,"Query,Create,Update,Delete,Editing,Uploads",JoelArcGISImport,GISPublisher,dev-bld-sql19\sde,C:\Users\ah4236\AppData\Local\Temp\ArcGISProTe...,Lopez-3291


# Delete Services

In [7]:
delete_services(delete_list)

Bedford_Map_Templates_test DELETED
Broadband_Inspection_Viewer_FeatureClasses DELETED
Line1 DELETED
NativeSQLPoly DELETED
Poly1 DELETED
SQLGeogpt DELETED
SQLGeom DELETED
SQLGeomTesting DELETED
WR_Polygon DELETED
WR_Pt DELETED
